# Steam Scraping

## Setup

In [26]:
# Installs
%pip install selenium
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.0 MB 9.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.7/11.0 MB 9.1 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.0 MB 9.7 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 9.9 MB/s eta 0:00:01
   ------------------------------------- -- 10.2/11.0 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 9.8 MB/s  0:00:01
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.8 MB 9.9 MB/s eta 0:00:02
   ------------- -------------------------- 4.5/12.8 MB 10.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.8 MB 10.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.8 MB 11.0 MB/s eta 0:00:01
   --------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [82]:
# Save Installs
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [84]:
# Imports
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import pandas as pd
import os

In [ ]:
# Globals
URL = "https://store.steampowered.com/search/?filter=topsellers"
TARGET_GAMES = 300
pause = 1

## Game ID Scraping

### Selenium

In [73]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

In [ ]:
driver.get(URL)
time.sleep(pause)
WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.ID, "search_resultsRows")))

apps = []
app_ids = []
last = None
height = 0

while True:
	# Load More
	if last:
		last = apps[-1]
		driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(pause)
		print("[LOG] Loading More,", len(apps))

	rows = driver.find_elements(By.CSS_SELECTOR, "#search_resultsRows a[data-ds-appid]")

	for row in rows:
		id = row.get_attribute("data-ds-appid")
		if id in app_ids: continue

		app_ids.append(id)
		temp = row.find_elements(By.CSS_SELECTOR, ".title")
		title = temp[0].text if temp else ""
		apps.append({"App ID": id, "App Title": title})
		
		if len(apps) >= TARGET_GAMES:
			print(f"[LOG] Target {len(apps)} Reached")
			break
	
	# Exit Conditions
	if len(apps) >= TARGET_GAMES: break
	elif last and last == apps[-1]:
		print(f"[LOG] Reached App Max ({len(apps)})")
		break

	if last == None:
		last = apps[-1]



[LOG] Loading More,  50
[LOG] Loading More,  100
[LOG] Loading More,  149
[LOG] Loading More,  199
[LOG] Loading More,  249
[LOG] Loading More,  299
[LOG] Target 300 Reached


In [81]:
print(len(apps), "Games")
print(pd.DataFrame(apps))

300 Games
      App ID                                          App Title
0    1675200                                         Steam Deck
1    2622380                              ELDEN RING NIGHTREIGN
2    3531720         ELDEN RING NIGHTREIGN The Forsaken Hollows
3    1808500                                        ARC Raiders
4    1085660                                          Destiny 2
..       ...                                                ...
295     4000                                        Garry's Mod
296   457140                                Oxygen Not Included
297  1040200                                Crime Scene Cleaner
298   714010                                            Aimlabs
299  1250410  Microsoft Flight Simulator (2020) 40th Anniver...

[300 rows x 2 columns]


### Save Data

In [85]:
df = pd.DataFrame(apps)
os.makedirs("data", exist_ok=True)
df.to_csv("data/app_ids.csv", index=False)

# New Section